In [8]:
import nbimporter
from dice_rollers import rollSixStats
from dice_rollers import findStatModifier
from dice_rollers import rollxdy
import matplotlib.pyplot as plt

In [1]:
class Weapon(object):
    def __init__(self, damage_die: str = "1d6", statistic: str = "str", modifier: int = 0):
        self.damage_die = damage_die
        self.statistic = statistic
        self.modifier = modifier
    
    def dealDamage(self, is_crit: bool = False):
        if is_crit:
            return rollxdy(self.damage_die) + rollxdy(self.damage_die) + self.modifier
        else:
            return rollxdy(self.damage_die) + self.modifier

class Character(object):
    def __init__(self, name: str, hit_die: str = "1d10"):
        self.name = name
        self.hit_die = hit_die
        self.state = "alive"
        self.armorClass = 15
    
    def setStats(self, stat_to_modify: str, stat_number: int):
        if stat_to_modify == "str":
            self.str = stat_number
        elif stat_to_modify == "dex":
            self.dex = stat_number
        elif stat_to_modify == "con":
            self.con = stat_number
        elif stat_to_modify == "int":
            self.int = stat_number
        elif stat_to_modify == "wis":
            self.wis = stat_number
        elif stat_to_modify == "cha":
            self.cha = stat_number
    
    def rollForInitiative(self):
        self.initiative = rollxdy("1d20") + self.dex_mod
    
    def setModifiers(self):
        if self.str:
            self.str_mod = findStatModifier(self.str)
        if self.dex:
            self.dex_mod = findStatModifier(self.dex)
        if self.con:
            self.con_mod = findStatModifier(self.con)
        if self.int:
            self.int_mod = findStatModifier(self.int)
        if self.wis:
            self.wis_mod = findStatModifier(self.wis)
        if self.cha:
            self.cha_mod = findStatModifier(self.cha)
    
    def setHitpoints(self):
        self.hitpoints = rollxdy(self.hit_die) + self.con_mod
    
    def getAttacked(self, attack_dictionary: dict):
        print("%s gets attacked" % self.name)
        if attack_dictionary['to_hit'] >= self.armorClass:
            print("it's a hit")
            self.takeDamage(attack_dictionary['damage'])
        else:
            print("it's a miss")
    
    def takeDamage(self, damage: int):
        if damage >= self.hitpoints:
            self.state = "dead"
        self.hitpoints -= damage
        
    def equipWeapon(self, weapon: Weapon):
        self.weapon = weapon
    
    def attack(self, attack_stat: int = "str"):
        if attack_stat == "str":
            modifier = self.str_mod
        elif attack_stat == "dex":
            modifier = self.dex_mod
        elif attack_stat == "int":
            modifier = self.int_mod
        elif attack_stat == "wis":
            modifier = self.wis
        elif attack_stat == "cha":
            modifier = self.cha
        else:
            modifier = 0
        
        roll = rollxdy("1d20")
        
        to_hit = roll + modifier
        
        if roll == 20:
            damage = self.weapon.dealDamage(is_crit = True)
        elif roll != 20:
            damage = self.weapon.dealDamage(is_crit = True)
        else:
            damage = 0
        
        damage += modifier
        
        return {'to_hit': to_hit,
               'damage': damage}
        

In [6]:
def createCharacterWithStatList(name: str, stat_list: list, stat_rank_order: list):
    """
    Creates a Character with given name.
    Stats are taken from stat_list and assigned 
    in descending rank order based on stat_rank_order
    """
    stat_list.sort(reverse=True)
    character = Character(name)
    for i in range(len(stat_rank_order)):
        character.setStats(stat_to_modify = stat_rank_order[i], stat_number = stat_list[i])
    character.setModifiers()
    character.setHitpoints()
    return character

In [7]:
stat_list = rollSixStats()
stat_rank_order = ['str',
                  'dex',
                  'con',
                  'int',
                  'wis',
                  'cha']
Dan = createCharacterWithStatList("Dan", stat_list, stat_rank_order)
Dan.setHitpoints()
katana = Weapon()
Dan.equipWeapon(katana)
Dan.attack()

{'to_hit': 21, 'damage': 12}

In [23]:
# let's find the roll distribution, so that we can use it to set up initial states

generated_rolls: list = []
num_samples: int = 10000
roll_distribution: list = []
    
# create a list of rolls 
for i in range(num_samples):
    roll = rollSixStats()[0]
    generated_rolls.append(roll)
    i += 1

for i in range(1, 19):
    frequency = generated_rolls.count(i) / len(generated_rolls)
    roll_distribution.append(frequency)

In [ ]:
# crate a function to fight two characters and return the winner
def fightToTheDeath(character_1: Character, character_2: Character):
    # set initiative
    character_1.rollForInitiative()
    character_2.rollForInitiative()
    
    # get the characters in order
    if character_2.initiative > character_1.initiative:
        character_1, character_2 = character_2, character_1
        remember_to_swap = True
    
    while (character_1.status == "alive" and character_2.status == "alive"):
        # define logic to get characters to fight one another